In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
from models.ddpm_unet_cattn import SPVUnet
import torch
import lightning as L
from models.g_spvd import GSPVD

/opt/conda/envs/spvd/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
model_args = {
    'voxel_size' : 0.1,
    'nfs' : (32, 64, 128, 256), 
    'attn_chans' : 8, 
    'attn_start' : 3, 
    'cross_attn_chans' : 8, 
    'cross_attn_start' : 2, 
    'cross_attn_cond_dim' : 768,
}

model = SPVUnet(**model_args)
model = GSPVD(model=model, lr=0.1, training_steps=1)

In [18]:
from dataloaders.shapenet.shapenet_loader import ShapeNet

categories = ['car']
path = "../data/ShapeNet"

te = ShapeNet(path, "val", 2048, categories, load_renders=True)

Loading (val) renders for bowl (02880940): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 38.04it/s]


In [19]:
import numpy as np

samples = 16
references = [te[idx] for idx in np.random.choice(list(range(len(te))), size=(samples,))]

reference_images = torch.stack([r["render-features"] for r in references]).to("cuda")

In [47]:
steps = 125
scheduler = 'ddpm'
ckpt_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/{steps}-steps.ckpt'
ckpt = torch.load(ckpt_path, weights_only=True)
# ckpt_path = '../checkpoints/distillation/GSPVD/starting.ckpt'
# ckpt = torch.load(ckpt_path, weights_only=True)['state_dict']
model.load_state_dict(ckpt)

<All keys matched successfully>

In [48]:
model = model.cuda().eval()

In [49]:
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler
from my_schedulers.ddim_scheduler import DDIMSparseScheduler
from utils.visualization import display_pointclouds_grid

if scheduler == 'ddim':
    sched = DDIMSparseScheduler(beta_min=0.0001, beta_max=0.02, steps=steps, init_steps=1000)
else:
    sched = DDPMSparseScheduler(beta_min=0.0001, beta_max=0.02, steps=steps, init_steps=1000)

In [ ]:
preds = sched.sample(model, samples, 2048, reference=reference_images)

Sampling:   3%|█████▏                                                                                                                                                              | 4/125 [00:00<00:06, 19.45it/s]

tensor(4.6156, device='cuda:0')
tensor(4.7315, device='cuda:0')
tensor(4.7525, device='cuda:0')
tensor(5.1452, device='cuda:0')


Sampling:   6%|██████████▍                                                                                                                                                         | 8/125 [00:00<00:05, 19.52it/s]

tensor(5.1952, device='cuda:0')
tensor(5.3107, device='cuda:0')
tensor(5.7945, device='cuda:0')
tensor(4.9846, device='cuda:0')


Sampling:  10%|███████████████▋                                                                                                                                                   | 12/125 [00:00<00:05, 19.56it/s]

...

Sampling:  90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍               | 113/125 [00:05<00:00, 20.20it/s]

tensor(3.1998, device='cuda:0')
tensor(3.2203, device='cuda:0')
tensor(3.1564, device='cuda:0')
tensor(3.2254, device='cuda:0')
tensor(2.9842, device='cuda:0')


Sampling:  95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏       | 119/125 [00:06<00:00, 21.44it/s]

tensor(2.8528, device='cuda:0')
tensor(2.8025, device='cuda:0')
tensor(2.6850, device='cuda:0')
tensor(2.6433, device='cuda:0')
tensor(2.6214, device='cuda:0')


Sampling: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:06<00:00, 19.86it/s]

In [ ]:
display_pointclouds_grid(preds.cpu().numpy(), offset=8, point_size=0.3)

In [ ]:
real = torch.stack([r["pc"] for r in references]).numpy()
display_pointclouds_grid(real, offset=8, point_size=0.3)